In [93]:
import pandas as pd
import numpy as np
import matplotlib as mbl
import matplotlib.pyplot as plt
import os
import shutil


directories = ['data_exes_004/data', 'data_exes_0005/data']

check = True

for directory in directories:

    for filename in os.listdir(directory):

        f = os.path.join(directory, filename)
        
        if os.path.isfile(f):

            if check:

                columns = 'L TEMP SUM SUME SUME2 VARE SUMM SUMAM SUMM2 VARM DELTAE DELTAT'
                data = pd.read_csv(f, header = None, delim_whitespace=True, names = columns.split(' '))
                check = False

            else:

                new_data = pd.read_csv(f, header = None, delim_whitespace=True, names = columns.split(' '))
                data = pd.concat([data, new_data])


data = data.sort_values(by=['L','TEMP']).reset_index(drop = True)

data.to_excel('processed_data.xlsx', sheet_name='imported_data')

data_matrix = np.array(data)

empty = np.array([[]])
with open("processed_data.txt", "ab") as f:

    for L in range(12,84,12):
        
        data_matrix_byL = data_matrix[data_matrix[:, 0] == L, :]
        np.savetxt(f, data_matrix_byL)
        np.savetxt(f, empty)
        np.savetxt(f, empty)



In [94]:
data.head()

,L,TEMP,SUM,SUME,SUME2,VARE,SUMM,SUMAM,SUMM2,VARM,DELTAE,DELTAT
0,12,1.64,500000.0,-276.055968,76320.155456,113.257988,18.273216,140.565136,19770.803616,19436.893193,0.000000,0.04
1,12,1.68,500000.0,-274.295728,75370.506432,132.360033,5.607464,140.001640,19615.646192,19584.202539,1.760240,0.04
2,12,1.72,500000.0,-272.314928,74308.737472,153.317460,12.545600,139.357440,19439.301120,19281.909041,0.000000,0.04
3,12,1.76,500000.0,-270.154472,73160.243616,176.804874,2.535456,138.627096,19241.053776,19234.625239,2.160456,0.04
4,12,1.80,500000.0,-267.727424,71882.548096,204.574534,-7.372244,137.779916,19013.111720,18958.761738,0.000000,0.04


In [95]:
data.shape

(720, 12)

In [96]:

import statsmodels.api as sm

data['cv'] = (data['SUME2'] - data['SUME']**2)/((data['TEMP']*(data['L']))**2)
data['x'] = (data['SUMM2'] - data['SUMAM']**2)/(data['TEMP']*data['L']**2)

inverseL = [1/i for i in range(12,84,12)]
critical_T_cv = []
critical_T_x = []

for i in range(12,84,12):

    critical_T_cv.append(data['TEMP'].iloc[data.loc[data['L'] == i].cv.idxmax()])
    critical_T_x.append(data['TEMP'].iloc[data.loc[data['L'] == i].x.idxmax()])

inverseL = sm.add_constant(inverseL)
fit1 = sm.OLS(critical_T_x, inverseL).fit()
fit2 = sm.OLS(critical_T_cv, inverseL).fit()
print(fit1.summary())
print(fit2.summary())



                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     2484.
Date:                Wed, 28 Dec 2022   Prob (F-statistic):           9.70e-07
Time:                        00:55:24   Log-Likelihood:                 28.874
No. Observations:                   6   AIC:                            -53.75
Df Residuals:                       4   BIC:                            -54.17
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2675      0.002   1322.964      0.0

c:\Users\ptbad\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
c:\Users\ptbad\anaconda3\lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [97]:
data

,L,TEMP,SUM,SUME,SUME2,VARE,SUMM,SUMAM,SUMM2,VARM,DELTAE,DELTAT,cv,x
0,12,1.64,500000.0,-276.055968,7.632016e+04,113.257988,18.273216,140.565136,19770.803616,19436.893193,0.000000,0.04,0.292428,0.051855
1,12,1.68,500000.0,-274.295728,7.537051e+04,132.360033,5.607464,140.001640,19615.646192,19584.202539,1.760240,0.04,0.325669,0.062777
2,12,1.72,500000.0,-272.314928,7.430874e+04,153.317460,12.545600,139.357440,19439.301120,19281.909041,0.000000,0.04,0.359892,0.075925
3,12,1.76,500000.0,-270.154472,7.316024e+04,176.804874,2.535456,138.627096,19241.053776,19234.625239,2.160456,0.04,0.396375,0.093048
4,12,1.80,500000.0,-267.727424,7.188255e+04,204.574534,-7.372244,137.779916,19013.111720,18958.761738,0.000000,0.04,0.438474,0.114994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,72,2.74,500000.0,-4875.045744,2.378791e+07,21842.837299,-1.021168,264.767824,109851.019568,109849.976784,120.079920,0.04,0.561233,2.798407
716,72,2.78,500000.0,-4762.162000,2.269938e+07,21195.032316,1.251480,250.164256,98016.079840,98014.513638,0.000000,0.04,0.529029,2.458722
717,72,2.82,500000.0,-4655.219872,2.169174e+07,20667.796520,0.014292,237.373788,88369.675608,88369.675404,106.942128,0.04,0.501338,2.190548
718,72,2.86,500000.0,-4553.598904,2.075540e+07,20134.306194,0.006132,225.131644,79577.826728,79577.826690,0.000000,0.04,0.474832,1.948813


In [100]:
print(critical_T_cv)
print(critical_T_x)

[2.33, 2.305, 2.28, 2.285, 2.28, 2.28]
[2.44, 2.35, 2.325, 2.31, 2.305, 2.295]


In [102]:
critical_df = pd.DataFrame([i for i in range(12,84,12)], columns=['L'])

critical_df['1/L'] = [1/i for i in range(12,84,12)]
critical_df['Tc'] = critical_T_cv
critical_df['Tx'] = critical_T_x 
critical_df['cv'] = [data.cv.iloc[data.loc[data['L'] == i].cv.idxmax()] for i in range(12,84,12)]
critical_df['x'] = [data.x.iloc[data.loc[data['L'] == i].x.idxmax()] for i in range(12,84,12)]

critical_df

,L,1/L,Tc,Tx,cv,x
0,12,0.083333,2.330,2.440,1.406713,3.653469
1,24,0.041667,2.305,2.350,1.762681,12.495912
2,36,0.027778,2.280,2.325,1.955083,25.672626
3,48,0.020833,2.285,2.310,2.109669,42.268911
4,60,0.016667,2.280,2.305,2.214874,62.909660
5,72,0.013889,2.280,2.295,2.315603,86.337807
